# Lab.4: Part of Speech
## Introduction to Human Language Technologies
### Victor Badenas Crespo

***

### Statement:

Given the following (lemma, category) pairs:
```python
(’the’,’DT’), (’man’,’NN’), (’swim’,’VB’), (’with’, ’PR’), (’a’, ’DT’),
(’girl’,’NN’), (’and’, ’CC’), (’a’, ’DT’), (’boy’, ’NN’), (’whilst’, ’PR’),
(’the’, ’DT’), (’woman’, ’NN’), (’walk’, ’VB’)
```

- For each pair, when possible, print their most frequent WordNet synset, their corresponding least common subsumer (LCS) and their similarity value, using the following functions:

    - Path Similarity

    - Leacock-Chodorow Similarity

    - Wu-Palmer Similarity

    - Lin Similarity

- Normalize similarity values when necessary. What similarity seems better?

*** 

## Solution

Import necessary packages and declare environment valiables.

In [24]:
import nltk
import numpy as np
from nltk.corpus import wordnet as wn

DATA = [
    ('the','DT'), ('man','NN'), ('swim','VB'), ('with', 'PR'), ('a', 'DT'),
    ('girl','NN'), ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), ('whilst', 'PR'),
    ('the', 'DT'), ('woman', 'NN'), ('walk', 'VB')
]

print(list(filter(lambda x: x[1].lower()[0] in ('v', 'n'), DATA)))

[('man', 'NN'), ('swim', 'VB'), ('girl', 'NN'), ('boy', 'NN'), ('woman', 'NN'), ('walk', 'VB')]


In [7]:
synsets = list()
for word, posTag in DATA:
    posTag = posTag.lower()[0]
    if posTag in ('n', 'v'):
        synset = wn.synsets(word, posTag)[0]
        synsets.append(synset)

In [26]:
distances = np.zeros((len(synsets), len(synsets)))
for ix in range(0, len(distances)):
    for iy in range(0, len(distances)):
        distance = synsets[ix].wup_similarity(synsets[iy])
        distances[ix,iy] = distance
print(distances)
lcsIndexes = np.nanargmin(distances, axis=0)
print(lcsIndexes)

[[1.                nan 0.63157895 0.66666667 0.66666667        nan]
 [0.18181818 1.         0.16666667 0.18181818 0.18181818 0.33333333]
 [0.63157895        nan 1.         0.63157895 0.63157895        nan]
 [0.66666667        nan 0.63157895 1.         0.66666667        nan]
 [0.66666667        nan 0.94736842 0.66666667 1.                nan]
 [0.18181818 0.33333333 0.16666667 0.18181818 0.18181818 1.        ]]
[1 5 1 1 1 1]


In [27]:
PathSimilarities = list()
LCSimilaities = list()
WuPalmerSimilarities = list()
LinSimilarities = list()
for synset, lcsIndex in zip(synsets, lcsIndexes):
    lcssynset = synsets[lcsIndex]
    PathSimilarities.append(synset.path_similarity(lcssynset))
    LCSimilaities.append(synset.lch_similarity(lcssynset))
    WuPalmerSimilarities.append(synset.wup_similarity(lcssynset))
    LinSimilarities.append(synset.lin_similarity(lcssynset))

WordNetError: Computing the lch similarity requires Synset('man.n.01') and Synset('swim.v.01') to have the same part of speech.

In [ ]:
print(PathSimilarities)
print(LCSimilaities)
print(WuPalmerSimilarities)
print(LinSimilarities)

***

## Conclusions

***

### End of P4